In [1]:
import sys
sys.path.append("/media/tupv8/32f7b466-e4d4-4f6f-b2aa-1e5a8ce7663c/vinai/hfnet_reproduce/hfnet_silk/")
from model.dataset.hpatches import HpatchesDataset

In [2]:
from torch.utils.data import DataLoader

import numpy as np
import torch

from model.hfnet import HFNet

from evaluation.local_descriptor import evaluate

In [3]:
# Load dataset
dataset = HpatchesDataset(alteration='all', make_pairs=True, shuffle=True)

# Create DataLoader
dataloader = DataLoader(dataset, batch_size=1, shuffle=False)

In [ ]:
EXPER_PATH = '/media/tupv8/32f7b466-e4d4-4f6f-b2aa-1e5a8ce7663c/vinai/reproduce/model/'
MODEL_PATH = '/media/tupv8/32f7b466-e4d4-4f6f-b2aa-1e5a8ce7663c/vinai/hfnet_reproduce/hfnet_silk/'
models = { 
    'silk':{
        'sigmoid':'weight/results/last_model.pth',
        
    },
}

config_model= {
    'image_channels':1,
    # 'loss_weights': 'uncertainties',
    'loss_weights':{
        'global':1,
        'local':1,
        'detector':1
    },
    'local':{
        'detector_threshold': 0.001,
        'nms_radius': 4,
        'num_keypoints': 5000
    },
    'local_head': {
        'descriptor_dim': 128,
        'detector_grid': 8,
        'input_channels': 96
    },
    'global_head': {
        'n_clusters': 32,
        'intermediate_proj': 0,
        'dimensionality_reduction': 4096
    }
}

config = {
        'local':{
            'nms_radius':0,
            'detector_threshold': 0.001,
            'num_keypoints': 1000,
        },
        'do_ratio_test': True,
        'correct_match_thresh': 3,
        'correct_H_thresh': 3,
        'num_features': 300,
        # 'do_nms': True,
        # 'nms_thresh': 4,
        # 'remove_borders': 4,
    }
    


model = HFNet(config_model, width_mult=0.75)

In [9]:
import time

device = torch.device('cpu')
model.to(device)

for k,v in models.items():
    for kk,vv in v.items():
        print(f"{k}: {kk}")
        print(config)
        state_dict = torch.load(MODEL_PATH + vv, 
                                weights_only= False,
                                map_location=device)
        state_dict.pop("logvars", None) 
        model.load_state_dict(state_dict, strict=False)
        metrics,_,_=evaluate(model, dataloader, config)
        
        

        current_time = time.strftime("%Y-%m-%d %H:%M:%S", time.localtime())
        print(current_time)

        log_file = open('descriptors.txt', 'a')
        log_file.write('\n[{}]'.format(current_time))
        log_file.write('\n{}: {}'.format(k, kk))
        log_file.write('{}\n'.format(config))


        for km, vm in metrics.items():
                print('{:<25} {:.3f}'.format(km, vm))

                #save in log file
                log_file.write('{:<25} :{:.3f}  '.format(km, vm))
                log_file.write('\n')
        log_file.flush()
        print("-----------------------------------")

       

silk: sigmoid
({'local': {'nms_radius': 0, 'detector_threshold': 0.001, 'num_keypoints': 1000}, 'do_ratio_test': True, 'correct_match_thresh': 3, 'correct_H_thresh': 3, 'num_features': 300},)


  0%|          | 0/580 [00:00<?, ?it/s]


TypeError: tuple indices must be integers or slices, not str